In [ ]:
# Same for Tasic 2016 (SRP061902)
wget -O tasic2016_metadata.tsv "https://www.ebi.ac.uk/ena/portal/api/filereport?accession=SRP061902&result=read_run&fields=run_accession,fastq_ftp&format=tsv&download=true"

## Simple download

In [ ]:
cat tasic2016_metadata.tsv | tail -n +2 | while IFS=$'\t' read -r run_acc fastq_urls; do
    IFS=';' read -ra urls <<< "$fastq_urls"
    for url in "${urls[@]}"; do
        echo "wget -c 'ftp://$url'"
    done
done > download_commands_2016.sh

## Checks for existing files before adding them to the download list

In [ ]:
# Extract SRR numbers and create download commands, checking for existing files
cat tasic2016_metadata.tsv | tail -n +2 | while IFS=$'\t' read -r run_acc fastq_urls; do
    IFS=';' read -ra urls <<< "$fastq_urls"
    for url in "${urls[@]}"; do
        # Extract filename from URL
        filename=$(basename "$url")
        # Check if file exists and is not empty
        if [ ! -s "$filename" ]; then
            echo "wget -c 'ftp://$url'"
        else
            echo "# Skipping $filename - already exists" >&2
        fi
    done
done > download_commands_2016.sh

## Add an MD5 check if you want to verify existing files

In [ ]:
# Extract SRR numbers and create download commands, checking for existing files
cat tasic2016_metadata.tsv | tail -n +2 | while IFS=$'\t' read -r run_acc fastq_urls md5s; do
    IFS=';' read -ra urls <<< "$fastq_urls"
    IFS=';' read -ra checksums <<< "$md5s"
    
    for i in "${!urls[@]}"; do
        url="${urls[$i]}"
        md5="${checksums[$i]}"
        filename=$(basename "$url")
        
        # Check if file exists and has correct MD5
        if [ -f "$filename" ]; then
            existing_md5=$(md5sum "$filename" | cut -d' ' -f1)
            if [ "$existing_md5" = "$md5" ]; then
                echo "# Skipping $filename - already exists with correct MD5" >&2
                continue
            else
                echo "# $filename exists but MD5 mismatch - will redownload" >&2
            fi
        fi
        echo "wget -c 'ftp://$url'"
    done
done > download_commands_2016.sh

## Use HTTPS instead of FTP, which is generally more reliable

In [ ]:
cat tasic2016_metadata.tsv | tail -n +2 | while IFS=$'\t' read -r run_acc fastq_urls; do
    IFS=';' read -ra urls <<< "$fastq_urls"
    for url in "${urls[@]}"; do
        # Convert FTP URL to HTTPS
        https_url=$(echo "$url" | sed 's|^|https://ftp.sra.ebi.ac.uk/|')
        echo "wget -c '$https_url'" 
    done
done > download_commands_2016.sh

## Execute

In [ ]:
chmod +x download_commands.sh

parallel -j 32 < download_commands_2016.sh

In [ ]:
ls -R logs/ | head -n 10
cat -n 10 logs/SRR7318040/*_mapping.log
cat -n 10 logs/SRR7318040/gapless.log
cat -n 10 logs/SRR7318040/countit.log

In [ ]:
conda create -n quantas python=3.7
conda activate quantas
conda install -c conda-forge glibc=2.29
conda install -c chaolinzhanglab quantas

In [ ]:
#!/bin/bash

# Create necessary directories if they don't exist
mkdir -p cache gapless_out countit_out logs genome_index/olego

# First ensure we have the genome index and required files
if [ ! -f "genome_index/olego/mm10/mm10.fa" ]; then
    echo "Need to setup genome index first. Download mm10 and run: olegoindex -a bwtsw mm10.fa"
    exit 1
fi

# Function to process a single paired-end sample
process_sample() {
    local srr=$1
    local logdir="logs/${srr}"
    mkdir -p "$logdir"

    echo "Processing ${srr}..."

    # 1. Mapping reads with OLego - note using proper file paths as per documentation
    echo "Mapping ${srr} read 1..."
    olego -v -t 16 \
        -j mm10.intron.hmr.bed \
        -o ${srr}_1.sam \
        genome_index/olego/mm10/mm10.fa \
        tasic2018/raw_data/${srr}_1.fastq.gz \
        2> "${logdir}/${srr}_1_mapping.log"

    echo "Mapping ${srr} read 2..."
    olego -v -t 16 \
        -j mm10.intron.hmr.bed \
        -o ${srr}_2.sam \
        genome_index/olego/mm10/mm10.fa \
        tasic2018/raw_data/${srr}_2.fastq.gz \
        2> "${logdir}/${srr}_2_mapping.log"

    # 2. Process paired-end reads - note the correct file path for isoform database
    echo "Running gapless on ${srr}..."
    perl gapless/gapless_huge_file.pl -v -sam -uniq \
        --split-size 10000000 \
        -isoform mm10/mm10.exon.trio.hmr.nr.bed \
        -E 400 -big --print-singleton \
        --library-type unstranded \
        -o gapless_out/${srr} \
        ${srr}_1.sam ${srr}_2.sam \
        2> "${logdir}/${srr}_gapless.log"

    # 3. Quantify splicing - using mm10.conf from your mm10 directory
    echo "Quantifying splicing for ${srr}..."
    perl countit/summarize_splicing_wrapper.pl \
        -c ./cache -v -big -weight \
        -conf mm10/mm10.conf -dbkey mm10 \
        -cass -taca -alt5 -alt3 -mutx -iret \
        gapless_out/${srr}/pair.gapless.bed \
        countit_out/${srr} \
        2> "${logdir}/${srr}_countit.log"

    # Clean up intermediate files if they exist
    if [ -f "${srr}_1.sam" ]; then rm "${srr}_1.sam"; fi
    if [ -f "${srr}_2.sam" ]; then rm "${srr}_2.sam"; fi
}

# Get list of unique SRR IDs from your fastq files
srr_ids=$(ls tasic2018/raw_data/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.fastq.gz//' | sort | uniq)

# Process each sample
for srr in $srr_ids; do
    if ! process_sample $srr; then
        echo "Processing failed for ${srr}, check logs directory"
        exit 1
    fi
done

# After processing all samples, create visualization files
for srr in $srr_ids; do
    echo "Creating visualization files for ${srr}..."
    perl quantas/countit/tag2profile.pl -v -big -weight -c ./cache \
        -exact -of bedgraph -n "${srr}_profile" \
        gapless_out/${srr}/pair.gapless.bed \
        ${srr}.bedGraph

    perl quantas/countit/tag2junction.pl -v -big -weight -c ./cache \
        gapless_out/${srr}/pair.gapless.bed \
        ${srr}.junction.count.bed
done